In [2]:
import torch

from flair.data import Sentence
from flair.embeddings import DocumentPoolEmbeddings, FlairEmbeddings, BertEmbeddings, ELMoEmbeddings

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

## 1. Load embeddings

In [7]:
# [[flair_eu, bert_cased_eu, bert_uncased eu, elmo_eu], [flair_en, bert_cased_en, bert_uncased en, elmo_en]]
embeddings_all = [[], []]
embedding_names = ["flair", "bert_cased", "bert_uncased", "elmo"]

In [8]:
# load flair embeddings
embeddings_all[0].append(DocumentPoolEmbeddings([FlairEmbeddings('eu-forward'), FlairEmbeddings('eu-backward')]))
embeddings_all[1].append(DocumentPoolEmbeddings([FlairEmbeddings('mix-forward'), FlairEmbeddings('mix-backward')]))

In [9]:
# load BERT embeddings

# See BERT paper, section 5.3 and table 7
bert_layers = '-1,-2,-3,-4'
bert_type = 'base' # 'large'

# BERT cased
embeddings_all[0].append(DocumentPoolEmbeddings([BertEmbeddings('bert-base-multilingual-cased', layers=bert_layers)]))
embeddings_all[1].append(DocumentPoolEmbeddings([BertEmbeddings('bert-'+bert_type+'-cased', layers=bert_layers)]))

# BERT uncased
embeddings_all[0].append(DocumentPoolEmbeddings([BertEmbeddings('bert-base-multilingual-uncased', layers=bert_layers)]))
embeddings_all[1].append(DocumentPoolEmbeddings([BertEmbeddings('bert-'+bert_type+'-uncased', layers=bert_layers)]))

2019-07-14 17:55:10,508 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
2019-07-14 17:55:27,231 The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [10]:
# load ELMo embeddings
embeddings_all[0].append(DocumentPoolEmbeddings([ELMoEmbeddings(options_file="https://schweter.eu/cloud/eu-elmo/options.json", 
                                                                weight_file="https://schweter.eu/cloud/eu-elmo/weights.hdf5")]))
embeddings_all[1].append(DocumentPoolEmbeddings([ELMoEmbeddings()]))

## 2. Read file and extract sentences

In [3]:
def get_gold_sentences(filename):
    gold_sentences = []
    with open(filename, 'rt') as f_p:
        for line in f_p:
            if line.startswith('"origin"'): # header
                continue
            
            if not line:
                continue
            
            line = line.rstrip()
            line = line.replace('"', '')
            splitted = line.split('\t')
            gold = splitted[0]
            sim_sentences = splitted[1:11]
            
            if gold:
                gold_sentences_simple = {}
                gold_sentences_simple[gold] = sim_sentences
                gold_sentences.append(gold_sentences_simple)
            
    return gold_sentences

def initialize_vectors(sent):
    similarities_all = []
    for i in range(len(sent)):
        similarities_all.append([])

    scores_all = []
    for i in range(len(sent)):
        scores_all.append([])
        
    return similarities_all, scores_all

In [14]:
test_eu = "goldstandard_eu_lexicover.tsv"
test_en = "goldstandard_en_lexicover.tsv"

sent_eu = get_gold_sentences(test_eu)
sent_en = get_gold_sentences(test_en)

similarities_all_eu, scores_all_eu = initialize_vectors(sent_eu)
similarities_all_en, scores_all_en = initialize_vectors(sent_en)

## 3. Calculate similarities

In [5]:
def calculate_similarities(gold, sim_sentences, embeddings):
    
    similarities = []
    query = gold

    q = Sentence(query)
    embeddings.embed(q)
    score = 0
    
    for i in range(len(sim_sentences)):
        
        s = Sentence(sim_sentences[i])
        embeddings.embed(s)

        assert q.embedding.shape == s.embedding.shape
        
        cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
        prox = cos(q.embedding, s.embedding)
        similarities.append(round(prox.item(), 4))
        
        if i > 0 and similarities[i] <= similarities[i-1]:
            score += 1
        
    return similarities, int(score/float(len(sim_sentences))*100)

def calculate(gold_sentences, embeddings, similarities_all, scores_all):
    
    for i in range(len(gold_sentences)):
        
        # obtain gold sentence and similar sentences from global list
        gold = list(gold_sentences[i].keys())[0]
        sim_sentences = gold_sentences[i][gold]
        
        # Calculate similarities for each 'gold' sentence and accumulated score
        similarities, score = calculate_similarities(gold, sim_sentences, embeddings)

        # append current similarity values and score to the global data structures
        scores_all[i].append(score)
        similarities_all[i].append(similarities)

        
    return similarities_all, scores_all

In [15]:
for i in range(len(embeddings_all[0])):
    print(f"Calculating {embedding_names[i]} embeddings for basque")
    similarities_all_eu, scores_all_eu = calculate(sent_eu, embeddings_all[0][i], similarities_all_eu, scores_all_eu)
    print(f"Calculating {embedding_names[i]} embeddings for english")
    similarities_all_en, scores_all_en = calculate(sent_en, embeddings_all[1][i], similarities_all_en, scores_all_en)

Calculating flair embeddings for basque
Calculating flair embeddings for english
Calculating bert_cased embeddings for basque
Calculating bert_cased embeddings for english
Calculating bert_uncased embeddings for basque
Calculating bert_uncased embeddings for english
Calculating elmo embeddings for basque
Calculating elmo embeddings for english


## 4. Plot similarities

In [12]:
fig = plotly.tools.make_subplots(rows=1, cols=10)

def plot_similarities(sent, similarities_all, scores_all):
    origin = list(sent[i].keys())[0]
    # print origin sentence
    print(origin + '\n')
    for j in range(len(sent[i][origin])):
        # print each similar sentence
        print(f"{j}. {sent[i][origin][j]}")
        for k in range(len(embedding_names)):
            # print each similarity value for each variant
            print(f"\t {embedding_names[k]} similarity: {similarities_all[i][k][j]}")
    # print scores for all variants
    print(f"Scores: " + ", ".join(f"{embed}: {scor}%" for embed, scor in zip(embedding_names, scores_all[i])))
    
    # plot similarity heatmap
    trace = go.Heatmap(z=similarities_all[i], y=embedding_names, colorscale='Blues')
    data=[trace]
    fig.append_trace(trace, 1, i+1)
    iplot(data, filename='basic-heatmap' + str(i))

This is the format of your plot grid:
[ (1,1) x1,y1 ]     [ (1,2) x2,y2 ]     [ (1,3) x3,y3 ]     [ (1,4) x4,y4 ]     [ (1,5) x5,y5 ]     [ (1,6) x6,y6 ]     [ (1,7) x7,y7 ]     [ (1,8) x8,y8 ]     [ (1,9) x9,y9 ]     [ (1,10) x10,y10 ]



In [13]:
for i in range(min(len(sent_eu), len(sent_en))):
    
    print(f"\nSentence #{i}")
    print("Basque")
    plot_similarities(sent_eu, similarities_all_eu, scores_all_eu)
    print("English")
    plot_similarities(sent_en, similarities_all_en, scores_all_en)


Sentence #0
Basque
Jokinek Amaia bazkari batera gonbidatu zuen

0. Jokinek gonbidapen bat eskeini zion Amaiari bazkari baterako
	 flair similarity: 0.7616
	 bert_cased similarity: 0.9656
	 bert_uncased similarity: 0.9573
	 elmo similarity: 0.8089
1. Jokinek Amaiari galdetu zion, ea berarekin bazkaldu nahi zuen
	 flair similarity: 0.708
	 bert_cased similarity: 0.918
	 bert_uncased similarity: 0.9379
	 elmo similarity: 0.7141
2. Jokinek Amaiari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan?
	 flair similarity: 0.5921
	 bert_cased similarity: 0.8871
	 bert_uncased similarity: 0.9066
	 elmo similarity: 0.701
3. Amaiak galdera jaso zuen Jokinengandik, ea bazkaltzea joan nahiko lukeen
	 flair similarity: 0.5961
	 bert_cased similarity: 0.9011
	 bert_uncased similarity: 0.9212
	 elmo similarity: 0.6695
4. Jokinek Amaiari gonbidapen bat eskeini zion otordu baterako
	 flair similarity: 0.7694
	 bert_cased similarity: 0.9612
	 bert_uncased similarity: 0.948
	 elmo similarity: 0.7767
5. 

English
Jokin invited Amaia for lunch

0. Jokin offered an invitation to Amaia for lunch
	 flair similarity: 0.9032
	 bert_cased similarity: 0.9666
	 bert_uncased similarity: 0.9196
	 elmo similarity: 0.8786
1. Jokin asked Amaia, if she wanted to have lunch with him
	 flair similarity: 0.7602
	 bert_cased similarity: 0.9194
	 bert_uncased similarity: 0.8249
	 elmo similarity: 0.741
2. Amaia received the question from Jokin, if she would like to go for lunch
	 flair similarity: 0.7663
	 bert_cased similarity: 0.9206
	 bert_uncased similarity: 0.805
	 elmo similarity: 0.6745
3. Jokin offered an invitation to Amaia for a meal
	 flair similarity: 0.8483
	 bert_cased similarity: 0.9576
	 bert_uncased similarity: 0.8579
	 elmo similarity: 0.8191
4. Amaia received an invitation from Jokin
	 flair similarity: 0.8101
	 bert_cased similarity: 0.9267
	 bert_uncased similarity: 0.8061
	 elmo similarity: 0.6237
5. Jokin and Amaia met for lunch
	 flair similarity: 0.9063
	 bert_cased similarity: 0.9


Sentence #1
Basque
Jokinek Amaia bazkari batera gonbidatu zuen

0. Jokinek gonbidapen bat eskeini zion Amaiari bazkari baterako
	 flair similarity: 0.7616
	 bert_cased similarity: 0.9656
	 bert_uncased similarity: 0.9573
	 elmo similarity: 0.8092
1. Jokinek Amaiari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan?
	 flair similarity: 0.5921
	 bert_cased similarity: 0.8871
	 bert_uncased similarity: 0.9066
	 elmo similarity: 0.7005
2. Jokinek Amaiari gonbidapen bat eskeini zion otordu baterako
	 flair similarity: 0.7694
	 bert_cased similarity: 0.9612
	 bert_uncased similarity: 0.948
	 elmo similarity: 0.7769
3. Bazkaltzeko elkartu ziren Jokin eta Amaia
	 flair similarity: 0.5851
	 bert_cased similarity: 0.9237
	 bert_uncased similarity: 0.9377
	 elmo similarity: 0.6647
4. Amaiak Jokin gonbidatu zuen bazkari batera
	 flair similarity: 0.8692
	 bert_cased similarity: 0.9716
	 bert_uncased similarity: 0.9745
	 elmo similarity: 0.8944
5. Amaia eta Jokin lagunak dira
	 flair similarity

English
the doctor invited the patient for lunch

0. the doctor offered an invitation to the patient for lunch
	 flair similarity: 0.9411
	 bert_cased similarity: 0.9714
	 bert_uncased similarity: 0.9405
	 elmo similarity: 0.9233
1. the doctor asked the patient, if she wanted to have lunch with him
	 flair similarity: 0.8355
	 bert_cased similarity: 0.9458
	 bert_uncased similarity: 0.8764
	 elmo similarity: 0.8078
2. the patient received the question from the doctor, if she would like to go for lunch
	 flair similarity: 0.8439
	 bert_cased similarity: 0.9453
	 bert_uncased similarity: 0.8402
	 elmo similarity: 0.75
3. the doctor offered an invitation to the patient for a meal
	 flair similarity: 0.9028
	 bert_cased similarity: 0.9615
	 bert_uncased similarity: 0.9149
	 elmo similarity: 0.8879
4. the patient received an invitation from the doctor
	 flair similarity: 0.882
	 bert_cased similarity: 0.9414
	 bert_uncased similarity: 0.8219
	 elmo similarity: 0.7779
5. the doctor and the p


Sentence #2
Basque
Medikuak gaixoa bazkari batera gonbidatu zuen

0. Medikuak gonbidapen bat eskeini zion gaixoari bazkari baterako
	 flair similarity: 0.7725
	 bert_cased similarity: 0.966
	 bert_uncased similarity: 0.9645
	 elmo similarity: 0.8
1. Medikuak gaixoari galdetu zion: Nahiko zenuke nirekin bazkaltzera joan?
	 flair similarity: 0.6132
	 bert_cased similarity: 0.9028
	 bert_uncased similarity: 0.9161
	 elmo similarity: 0.7215
2. Medikuak gaixoari gonbidapen bat eskeini zion otordu baterako
	 flair similarity: 0.7877
	 bert_cased similarity: 0.9592
	 bert_uncased similarity: 0.9531
	 elmo similarity: 0.7989
3. Bazkaltzeko elkartu ziren medikua eta gaixoa
	 flair similarity: 0.6131
	 bert_cased similarity: 0.9158
	 bert_uncased similarity: 0.9367
	 elmo similarity: 0.6702
4. gaixoak medikua gonbidatu zuen bazkari batera
	 flair similarity: 0.9113
	 bert_cased similarity: 0.959
	 bert_uncased similarity: 0.9819
	 elmo similarity: 0.9068
5. gaixoa eta medikua lagunak dira
	 fla

...

English
Happiness isn't doing what you like, but liking what you do

0. Liking what you do makes you happy, not doing what you want
	 flair similarity: 0.9409
	 bert_cased similarity: 0.9627
	 bert_uncased similarity: 0.9295
	 elmo similarity: 0.8837
1. The source of happines is liking what you do
	 flair similarity: 0.8647
	 bert_cased similarity: 0.9297
	 bert_uncased similarity: 0.8105
	 elmo similarity: 0.7712
2. What is done gladly is happiness
	 flair similarity: 0.757
	 bert_cased similarity: 0.8839
	 bert_uncased similarity: 0.7383
	 elmo similarity: 0.6303
3. What you do, not what you like, is happiness
	 flair similarity: 0.9061
	 bert_cased similarity: 0.9449
	 bert_uncased similarity: 0.8806
	 elmo similarity: 0.8203
4. Happiness isn't liking what you do, but doing what you like
	 flair similarity: 0.9859
	 bert_cased similarity: 0.9936
	 bert_uncased similarity: 0.982
	 elmo similarity: 0.9712
5. Happiness is shown in actions
	 flair similarity: 0.6498
	 bert_cased similar


Sentence #9
Basque
Maite dudalako sortzen dut

0. Sortzen dut maite dudalako
	 flair similarity: 0.9097
	 bert_cased similarity: 0.9155
	 bert_uncased similarity: 0.9557
	 elmo similarity: 0.8887
1. Maitasunaren ondorioz sortzen dut
	 flair similarity: 0.7084
	 bert_cased similarity: 0.9378
	 bert_uncased similarity: 0.9115
	 elmo similarity: 0.7343
2. Maite dut: ondorioz, sortzen dut
	 flair similarity: 0.8645
	 bert_cased similarity: 0.9194
	 bert_uncased similarity: 0.9032
	 elmo similarity: 0.8793
3. Sortzearen arrazoia maitasuna da
	 flair similarity: 0.456
	 bert_cased similarity: 0.8521
	 bert_uncased similarity: 0.8541
	 elmo similarity: 0.4817
4. Nire sorkuntzaren iturria maitasuna da
	 flair similarity: 0.475
	 bert_cased similarity: 0.8726
	 bert_uncased similarity: 0.8694
	 elmo similarity: 0.5218
5. Maite nuelako sortu nuen
	 flair similarity: 0.8459
	 bert_cased similarity: 0.8679
	 bert_uncased similarity: 0.8563
	 elmo similarity: 0.8998
6. Sortu ondoren maitatu nuen
	

English
Because I love, I create

0. I create because I love
	 flair similarity: 0.9474
	 bert_cased similarity: 0.9562
	 bert_uncased similarity: 0.9107
	 elmo similarity: 0.8838
1. Due to love I create
	 flair similarity: 0.82
	 bert_cased similarity: 0.9227
	 bert_uncased similarity: 0.7754
	 elmo similarity: 0.7821
2. I love: therefore, I create
	 flair similarity: 0.947
	 bert_cased similarity: 0.9627
	 bert_uncased similarity: 0.8557
	 elmo similarity: 0.8903
3. The reason for creating is love
	 flair similarity: 0.7241
	 bert_cased similarity: 0.8808
	 bert_uncased similarity: 0.7024
	 elmo similarity: 0.6161
4. The origin of my creating is love
	 flair similarity: 0.6892
	 bert_cased similarity: 0.9043
	 bert_uncased similarity: 0.7564
	 elmo similarity: 0.6521
5. Because I loved I created
	 flair similarity: 0.873
	 bert_cased similarity: 0.9482
	 bert_uncased similarity: 0.8513
	 elmo similarity: 0.9031
6. After creating I loved
	 flair similarity: 0.7781
	 bert_cased similar


Sentence #10
Basque
Maite dudalako sortzen dut

0. Sortzen dut maite dudalako
	 flair similarity: 0.9097
	 bert_cased similarity: 0.9155
	 bert_uncased similarity: 0.9557
	 elmo similarity: 0.8889
1. Maite dut: ondorioz, sortzen dut
	 flair similarity: 0.8645
	 bert_cased similarity: 0.9194
	 bert_uncased similarity: 0.9032
	 elmo similarity: 0.8797
2. Nire sorkuntzaren iturria maitasuna da
	 flair similarity: 0.475
	 bert_cased similarity: 0.8726
	 bert_uncased similarity: 0.8694
	 elmo similarity: 0.523
3. Sortu ondoren maitatu nuen
	 flair similarity: 0.6666
	 bert_cased similarity: 0.8761
	 bert_uncased similarity: 0.8906
	 elmo similarity: 0.637
4. Sortzen dudalako maite dut
	 flair similarity: 0.9437
	 bert_cased similarity: 0.9301
	 bert_uncased similarity: 0.9745
	 elmo similarity: 0.9156
5. Zaindu maite duzun hori
	 flair similarity: 0.7212
	 bert_cased similarity: 0.8152
	 bert_uncased similarity: 0.8892
	 elmo similarity: 0.5985
6. Maite zaitut
	 flair similarity: 0.7247
	 

Exception: The (row, col) pair sent is out of range. Use Figure.print_grid to view the subplot grid. 

## 5. Calculate total scores

In [13]:
def calculate_total_score(scores_all):
    total_scores = [0] * len(scores_all[0])
    for i in range(len(scores_all)):
        total_scores[0] += scores_all[i][0]
        total_scores[1] += scores_all[i][1]
        total_scores[2] += scores_all[i][2]
        total_scores[3] += scores_all[i][3]
    print(f"Total scores: " + ", ".join(f"{embed}: {scor}" for embed, scor in zip(embedding_names, total_scores)))

In [14]:
calculate_total_score(scores_all_eu)
calculate_total_score(scores_all_en)

Total scores: flair: 6, bert_cased: 5, bert_uncased: 4, elmo: 6
Total scores: flair: 5, bert_cased: 6, bert_uncased: 6, elmo: 6
